**Note: Run the following two cells if you are working in Google Colab!**

In [ ]:
!git clone https://github.com/KTH-dESA/MJ2383.git
!mv MJ2383/* .
!rm -r MJ2383

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()
!conda install -c conda-forge glpk
!pip install otoole==0.10.0b0

Don't forget to run the imports!

In [ ]:
import pandas as pd
from plotly import express as px
# from tqdm import trange
import os

# MJ2383 Computer Lab 3

This lab aims to provide an inside view on what a supply curve is and how it can be generated while using OSeMOSYS. Reminder, a supply curve is a graphical representation of the law of supply. It slopes upward because the quantity supplied increases as price increases, with other things constant.

In this lab, we'll be using OSeMOSYS, but we'll be running it in the background using this Jupyter Notebook to control the input data, run the model and extract and visualise the results.

If you click on the **folder** icon in the left-hand menu, you will see the folder structure containing the OSeMOSYS models used in this lab.

In the `model` folder, you will find subfolders containing OSeMOSYS models of increasing complexity. Each subfolder contains a data folder in which you see a number of CSV (comma-separated value) files.  Each file relates to one parameter in OSeMOSYS. For example, you could see the `CapitalCost` of technologies by opening the respective file at ``model/gas/data/CapitalCost.csv``.

## Editing data
In Colab, we can open the csv files and see the data, but unfortunately we cannot edit it. Thus, we will use the following funcions to easily `show`, `edit`, `add`, and `remove` data from the modes. Please run the following cell to crete the functions, we will use them further ahead in the notebook.

In [ ]:
def show_data(model: str, dataset: str) -> pd.DataFrame:
    """Reads and returns the dataset specified as a pandas DataFrame
  
    Parameters
    ----------
    model: str
    Name of the model from which you want to read the dataset from. Options are
    'gas', 'gas_more', 'gas_solar', and 'gas_solar_wind'.
    dataset: str
    Name of dataset to be read and shown. For options see `model/<model_name>/data/`.

    Returns
    -------
    pd.DataFrame
    DataFrame representation of the dataset.
    """
    df = pd.read_csv(os.path.join('model', model, 'data', f'{dataset}.csv'))
    return df

def edit_data(model: str, dataset: str, technology: str,
              value: str) -> pd.DataFrame:
    """Edits the value of a specific technology in a dataset and saves it.

    Parameters
    ----------
    model: str
    Name of the model from which you want to edit the dataset from. Options are
    'gas', 'gas_more', 'gas_solar', and 'gas_solar_wind'.
    dataset: str
    Name of dataset to edit. For options see `model/<model_name>/data/`.
    technology: str
    Name of the technology to edit the value. For options see
    `model/<model_name>/data/TECHNOLOGY.csv`
    Value: str | int | float
    New value to use for the specified technology and dataset.

    Returns
    -------
    pd.DataFrame
    Overwrites the edited DataFrame and resturns it.
    """
    file = os.path.join('model', model, 'data', f'{dataset}.csv')
    df = pd.read_csv(file)
    df.loc[df['TECHNOLOGY']==technology, 'VALUE'] = value
    df.to_csv(file, index=False)
    return df

def add_data(model: str, dataset: str, data: dict) -> pd.DataFrame:
    """Adds a new row of data to a specific dataset and saves it.

    Parameters
    ----------
    model: str
    Name of the model from which you want to edit the dataset from. Options are
    'gas', 'gas_more', 'gas_solar', and 'gas_solar_wind'.
    dataset: str
    Name of dataset to edit. For options see `model/<model_name>/data/`.
    data: dict
    Dictionary containing the parameter names and values to add to the desired
    dataset. for example:
    {'REGION': 'SIMPLICITY', 'EMISSION': 'CO2', 'YEAR': 2014, 'VALUE': 50}. For
    the `EmissionsPenalty.csv` dataset.
    Value: str | int | float
    New value to use for the specified technology and dataset.

    Returns
    -------
    pd.DataFrame
    Adds the new data to the DataFrame and overwrites its csv.
    """
    file = os.path.join('model', model, 'data', f'{dataset}.csv')
    df = pd.read_csv(file)
    if set(data.keys()) != set(df.columns):
        raise ValueError('Wrong column names! the correct coumns of the dataset ' +
                         'are ' + str(list(df.columns)))
    l = len(list(data.keys())[0:-1])
    i = 0
    for idx, row in df.iterrows():
        for key in list(data.keys())[0:-1]:
            if row[key] == data[key]:
                i += 1
            if i == l:
                remove_data(model=model, dataset=dataset, index=idx)
                df = add_data(model=model, dataset=dataset, data=data)
                return df
            else:
                i = 0
    index = len(df)
    df = pd.concat([df, pd.DataFrame(data, index=[index])], axis=0)
    df.to_csv(file, index=False)
    return df

def remove_data(model: str, dataset: str, index: int):
    """Removes a row of data from a specific dataset and saves it.

    Parameters
    ----------
    model: str
    Name of the model from which you want to edit the dataset from. Options are
    'gas', 'gas_more', 'gas_solar', and 'gas_solar_wind'.
    dataset: str
    Name of dataset to edit. For options see `model/<model_name>/data/`.
    index: int
    Index number of the row to remove. Use first the `show_data()` function to
    see the relevant index number of the row.

    Returns
    -------
    pd.DataFrame
    Removes the data from the DataFrame and overwrites its csv.
    """
    file = os.path.join('model', model, 'data', f'{dataset}.csv')
    df = pd.read_csv(file)
    df.drop(index, inplace=True)
    df.to_csv(file, index=False)
    return df

## Units
The units of the different input data are:

Parameter | Unit
:-- | --:
Demand | PJ
Capacity | GW
Activity | PJ
Capital Cost | M\$/GW
Fixed Cost | M\$/GW
Variable Cost, Fossil Fuels Extraction/Import | M\$/PJ
Variable Cost, Renewables | M\$/GWh
Operational Life | Yr
Emission Activity Ratio | MtCO2/PJ
Annual Emission Limit | MtCO2
Emission Penalty | \$/tCO2

## Parameters

**If you are not yet familiar with OSeMOSYS parameters, open the [manual](https://osemosys.readthedocs.io/en/latest/manual/Structure%20of%20OSeMOSYS.html#parameters) in a browser window for reference throughout the lab.**

## Running a model

Running a model is a two step process. We will explain the steps here, and use them later in the notebook. Firstly, you need to create an OSeMOSYS datafile with the next line following:

    !otoole convert datapackage datafile model/gas temp.txt
    
Note you need the `!` prepended to the command (this tells the notebook to run this command using the shell rather than Python). Here we create datafile called `temp.txt` from the model data stored in `model/gas`.

After generating the OSeMOSYS datafile (in this case, called `temp.txt`) we then solve the model using `glpsol`, which is the open-source solver provided by the GNU Math Programming kit.

    !glpsol -d temp.txt -m osemosys.txt > osemosys.log
    
The results are then available in the `results` folder. You should also check the `osemosys.log` file to ensure that the model ran correctly.

In the stages below, we load data from the model inputs and results using a Python library called `pandas` and plot them in interactive charts.

## Contents

- [Stage 1](#s1) - In this section, we explore a very simple model with a two-stage supply curve for natural gas to develop our economic interpretation of OSeMOSYS
- [Stage 2](#s2) - We add complexity, by increasing the number of steps in our supply curve by adding resources. We explore what difference this makes to the electricity price under different conditions.
- [Stage 3](#s3) - We add an emissions penalty, imposing a tax upon CO2
- [Stage 4](#s4) - We add renewable technologies to the electricity sector, whose marginal cost of generation is 0. However, this creates a demand for backup capacity. How does this affect the marginal price of electricity?

<a name="s1"></a>

## Stage 1: A super simple supply curve
We start with the simplest supply cost curve you can imagine. In this simple OSeMOSYS model, there are two natural gas commodities (fuels) - imports of natural gas `GasImport`, and extraction `GasExtraction`. Both of these generate CO2, and feed a natural gas combined-cycle electricity generation plant `NGCC`. This produces secondary electricity `SEC_EL` which enters a transmission and distribution technology `TD` which produces final electricity `FEL`.

![](https://github.com/KTH-dESA/MJ2383/blob/main/img/simple.svg?raw=1)

(Note, this image is produced using the command `!otoole viz res model/gas/datapackage.json img/simple.svg`)

---

First, let's have a think about the supply curve in this system.

Shown on the left of the image, there are two natural gas resources. `GasExtraction` has a maximum production capacity `TotalAnnualMaxCapacity` (`model/gas/data/TotalAnnualMaxCapacity.csv`) of 6 PJ/year at a variable cost `VariableCost` (`model/gas/data/VariableCost.csv`) of €8/PJ. However, `GasImport` has no upper bound, but a higher cost of €12/PJ.


In [ ]:
 show_data(model='gas', dataset='TotalAnnualMaxCapacity')

In [ ]:
 show_data(model='gas', dataset='VariableCost')

Plotted, this looks rather uninspiring, but at least gives an impression of a supply curve for natural gas.

__Run the next cell to view the graph__

In [ ]:
gas_extraction = [["GasExtraction", x, 8] for x in range(7)]
gas_import = [["GasImport", x, 12] for x in range(6, 21)]
gas_supply_curve = pd.DataFrame(gas_extraction + gas_import,
    columns=['name', 'supply', 'marginal_cost'])
fig = px.line(gas_supply_curve, x='supply', y='marginal_cost', range_x=[0, 20], range_y=[0, 20], line_shape='vh')
fig.update_traces(mode="markers+lines")
fig.update_xaxes(showspikes=True)
fig.show()

In OSeMOSYS, supply of energy must equal demand for energy. As the quantity of energy demanded increases, energy supply increases accordingly. Given that our demand curve is a straight line (imagine a vertical line moving along the x-axis) the equilibrium point (where the lines cross) denotes the marginal cost of production and in this case is equal to the price.

Within most energy systems, there are multiple markets for different, related, energy commodities. Later, we will use our simple OSeMOSYS model to begin to understand the interactions between these different energy markets.

### Exercise 1: Manually computing the marginal cost of electricity

Think about how our simple supply curve for natural gas interacts with the demand for electricity.

In our simple example, we demand a specific quantity of electricity. The electricity is produced by the `NGCC` technology which requires almost 2 units of gas to produce 1 unit of electricity.

**Now answer the quiz question 1 on Canvas.**

### Marginal Prices

In OSeMOSYS, there is a key equation called a "balancing constraint". This ensures that energy is conserved in each timeslice.

```ampl
s.t. EBa11_EnergyBalanceEachTS5{r in REGION, l in TIMESLICE, f in FUEL, y in YEAR}:
	Production[r,l,f,y] >= Demand[r,l,f,y] + Use[r,l,f,y]
	+ sum{rr in REGION} Trade[r,rr,l,f,y] * TradeRoute[r,rr,f,y];
```

We can extract some useful information from this equation when we solve the optimisation problem to minimise costs.  These data are stored in the file `results/ProductionDual.csv`.

### Exercise 2: Understanding production

We'll now run OSeMOSYS from this Jupyter Notebook to compute the equilibrium price. But first we will add some demand data to the `SpecifiedAnnualDemand` dataset:

In [ ]:
data = {"REGION": 'SIMPLICITY',"FUEL": 'FEL',"YEAR": 2014, "VALUE": 7}
add_data(model='gas', dataset='SpecifiedAnnualDemand', data=data)

Now we run OSeMOSYS and plot the total production by fuel over the year.  

In [ ]:
!otoole convert datapackage datafile model/gas temp.txt
# Solve the model, writing the results into ./results
!glpsol -d temp.txt -m osemosys.txt > osemosys.log
production = pd.read_csv('results/ProductionByTechnologyAnnual.csv').groupby(by=['TECHNOLOGY', 'FUEL']).sum()
px.bar(production.reset_index(), x='FUEL', y='VALUE', color='TECHNOLOGY', labels={'VALUE': 'Energy (PJ)'})

In reality, however, production differs across the year - in OSeMOSYS we use `TIMESLICES` to represent the fractions of the year in which different levels of demand exist. This approximates the average demand profile for electricity and other fuels across days and seasons. In our example model, we have defined 6 time-slices:

TIMESLICE | Description
:--|:--
ID | Intermediate day
IN | Intermediate night
SD | Summer day
SN | Summer night
WD | Winter day
WN | Winter night

In [ ]:
production = pd.read_csv('results/ProductionByTechnology.csv').groupby(by=['TIMESLICE', 'FUEL']).sum()
fig = px.bar(production.reset_index(), x='TIMESLICE', y='VALUE', color='FUEL',
             category_orders={"FUEL": ["GAS", "SEC_EL", "FEL"]}, labels={'VALUE': 'Energy (PJ)'})
fig.update_layout(barmode='group')

**Now answer quiz question 2 on Canvas**

### Exercise 3: Using OSeMOSYS to compute the marginal price

Below, we use a Python library called `pandas` to read the `ProductionDual` comma-separated values file. We extract the data point for `GAS` during the winter day (`WD`) timeslice.

In [ ]:
cost = pd.read_csv('results/ProductionDual.csv').set_index(['TIMESLICE', 'FUEL'])
cost['DUAL'] = cost['DUAL'] / (1.05**-0.5)
cost.loc[('WD', 'GAS'),'DUAL']

By running the cell, we see that the marginal cost of producing gas, is equal to the cost of importing natural gas: about €12/PJ.  Interesting.  That must mean that the model is importing gas, rather than extracting it (which is cheaper).

NB: The division of `(1.05**-0.5)` compounds the marginal value back to the base year. In OSeMOSYS, fixed and variable operating costs are discounted to the middle of the year.

**Now answer quiz question 3 on Canvas. Hint: Consider tweaking and re-running selected parts of above code.**

Plotted onto the supply curve we plotted earlier, we see that the results from the OSeMOSYS model make sense. The marginal cost of gas production from the OSeMOSYS model sits on the supply cost curve of the two gas resources.

However, we only have one point for electricity. In the next stages, we will use the model to compute the marginal cost of electricity production.

In [ ]:
fig = px.line(gas_supply_curve, x='supply', y='marginal_cost',
              range_x=[0, 20], range_y=[0, 30], line_shape='vh')

gas_demand = production.groupby(by='FUEL').sum().loc['GAS', 'VALUE']
elec_demand = production.groupby(by='FUEL').sum().loc['FEL', 'VALUE']

fig.add_shape( # adds a vertical "demand" line
    type="line", line_color="salmon", line_width=3, opacity=1, line_dash="dot",
    x0=gas_demand, x1=gas_demand, xref="x", y0=0, y1=cost.loc[('WD', 'GAS'),'DUAL'])

fig.add_annotation( # adds a text callout with arrow
    text="Marginal cost of gas", x=gas_demand, y=cost.loc[('WD', 'GAS'),'DUAL'],
    arrowhead=1, showarrow=True
)

fig.add_annotation( # adds a text callout with arrow
    text="Marginal cost of electricity", x=elec_demand,
    y=cost.loc[('WD', 'FEL'),'DUAL'], arrowhead=1, showarrow=True
)

fig.add_shape( # adds a vertical "demand" line
    type="line", line_color="salmon", line_width=3, opacity=1, line_dash="dot",
    x0=elec_demand, x1=elec_demand, xref="x", y0=0, y1=cost.loc[('WD', 'FEL'),'DUAL'])

fig.show()

### Notes

- The x-axis shows the supply quantity of energy
- The marginal cost of electricity is plotted for reference against the quantity of electricity supplied, and marginal cost of gas against the quantity of gas supplied

### Running the model to extract the marginal cost of production of electricity

We can automate the running of the OSeMOSYS model over multiple levels of demand for electricity. The following code loops over demands ranging from 1 to *n* in increments of *x*.  We write the demand into the CSV file `SpecifiedAnnualDemand.csv`, create the modelfile and then solve the model. We then extract the results from the `results/ProductionDual.csv` file and store them in a list.  Finally, the function returns the list of results for further processing.

In [ ]:
def extract_result(param: str, production):
    """Read the marginal cost values from the results
    """
    marginal = pd.read_csv('results/ProductionDual.csv'
                           ).set_index(['TIMESLICE', 'FUEL'])
    marg_ann = pd.read_csv('results/ProductionDualAnnual.csv'
                           ).set_index(['FUEL'])
    # production = pd.read_csv('results/ProductionByTechnology.csv').groupby(by=['TIMESLICE', 'FUEL']).sum()
    try:
        demand = production.groupby(by='FUEL').sum().loc[param, 'VALUE']
    except KeyError:
        demand = 0
    try:
        value = marginal.loc[('WD', param), 'DUAL'] / (1.05**-0.5)
    except KeyError:
        try:
            value = marg_ann.loc[param, 'DUAL'] / (1.05**-0.5)
        except KeyError:
            value = 0
    observation = {'value': value, 'param': param, 'quantity': demand}
    return observation

def run_model(model: str, start: int, stop: int, step=int):
    """Run the model for a range of demand values

    Returns
    -------
    List
        A list of dual values for GAS, FEL and COA
    """
    results = []
    for dem in trange(start, stop, step):
        observation = {}
        data = {"REGION": 'SIMPLICITY', "FUEL": 'FEL', "YEAR": 2014, "VALUE": dem}
        add_data(model=model, dataset='SpecifiedAnnualDemand', data=data)
        path_to_model = os.path.join('model', model)
        !otoole convert datapackage datafile $path_to_model temp.txt
        !glpsol -d temp.txt -m osemosys.txt > osemosys.log
        production = pd.read_csv('results/ProductionByTechnologyAnnual.csv').groupby(by=['TECHNOLOGY', 'FUEL']).sum()
        fuels = pd.read_csv(os.path.join('model', model, 'data', 'FUEL.csv'))
        for fuel in fuels['VALUE'].values:
            if fuel != 'SEC_EL':
                results.append(extract_result(fuel, production))

    return results

In [ ]:
gas_results = run_model("gas", 1, 20, 1)

In [ ]:
gas_data = pd.DataFrame(gas_results)
px.line(gas_data, x='quantity', y='value', facet_col='param', range_x=[0, 20], range_y=[0, 40],
        line_shape='vh', labels={'value': 'Cost'})

In the plot above, we now see the supply cost curves derived from the OSeMOSYS model. Given the simple supply cost curve for gas, with two stages and a breakpoint between 4.19 and 6.29 (we know that the cost of gas production changes at 6), you can see that the marginal cost of producing 2 units of electricity is 16.77 and the marginal cost of producing 3 or more units is 25.16.

## Summary of Stage 1

In this first part of the lab we learned the following:

- A simple supply curve can be constructed in OSeMOSYS using one technology per resource step. Parameters `TotalAnnualMaxCapacity` and `VariableCost` may be used to define each step in the supply cost curve for a resource.
- The OSeMOSYS code can be extended to write out marginal costs for all commodities in the model. This is particularly useful to show the marginal cost of derived commodities, such as electricity, which are produced from a variety of different energy chains.
- By running our model at different levels of demand, we can use the model to compute the derived supply cost curve for each commodity.
- We can use OSeMOSYS to explore the relationship between the structure of the energy system, supply cost curves for primary resources, and supply cost curves for secondary and tertiary energy commodities.

<a name='s2'></a>

# Stage 2: Adding Resources

In this part of the lab, we add a number of resources to our model and explore how this affects the supply curve for electricity. We now move towards a slightly more realistic energy system than the simplified model we used to introduce the concepts in Stage 1.

![](https://github.com/KTH-dESA/MJ2383/blob/main/img/gasmore.svg?raw=1)

(Note, this image is produced using the command `!otoole viz res model/gas_more/datapackage.json img/simple.svg`)

As you can see from the figure above, this model contains multiple gas resources, plus imports, multiple coal resources, plus an import, and a new super-critical coal fired power station.

The prices and quantities of the resources are as follows:

Resource | Quantity available | Cost of extraction/import
:--|--:|--:
GasExtraction | 10 | 8.0
GasExtraction2 | 15 | 10.0
GasExtraction3 | 30 | 11.0
GasImport | - | 12.0
CoalExtraction | 10 | 4.0
CoalExtraction2 | 15 | 5.0
CoalExtraction3 | 30 | 5.5
CoalImport | - | 12.0

These prices and quantities are defined in `model/gas_more/data/VariableCost.csv` and `model/gas_more/data/TotalTechnologyModelPeriodActivityUpperLimit.csv` respectively.

The gas-fired (`NGCC`) and coal-fired (`SCC`) power plants have the following characteristics:

Plant | Parameter | Value
:--|:--|--:
NGCC | CapitalCost | 1100.0
SCC | CapitalCost | 1600.0
NGCC | FixedCost | 44.0
SCC | FixedCost | 56.0
NGCC | InputActivityRatio(GAS) | 1.992
SCC | InputActivityRatio(COA) | 2.120
NGCC | OutputActivityRatio(SEC_EL) | 1.0
SCC | OutputActivityRatio(SEC_EL) | 1.0
NGCC | ResidualCapacity | 30
SCC| ResidualCapacity |30

In [ ]:
show_data('gas_more', 'VariableCost')

In [ ]:
more_results = run_model("gas_more", 1, 60, 2)

In [ ]:
more_data = pd.DataFrame(more_results)
px.line(more_data, x='quantity', y='value', facet_col='param',
        line_shape='hv',
        labels={'value': 'Cost'})

We now have a much more complicated supply cost curve for all commodities. Four distinct steps are evident in the supply cost curve for gas (`GAS`), three/four for coal (`COA`), and seven in the the plot for final electricity (`FEL`). Each of these steps in the gas and coal plots correspond to one of the resources that we defined in this more complicated model. However, the supply cost curve for final electricity (`FEL`) is a function of the combination of the gas and coal cost curves, and depends on the blend of coal and gas used to supply electricity at each level of demand.

In [ ]:
production = pd.read_csv('results/ProductionByTechnologyAnnual.csv').groupby(by=['TECHNOLOGY', 'FUEL']).sum()
px.bar(production.reset_index(), x='FUEL', y='VALUE', color='TECHNOLOGY',
       labels={'VALUE': 'Energy (PJ)'})

### Exercise 4

We can confirm the outputs from the model run with maximum demand by viewing the production of the different technologies. In the plot above you can view the quantity of coal and gas extracted or imported, and the quantity of secondary electricity produced by the power plants and final electricity transmitted to meet the demands.

**Now, answer quiz question 4 on Canvas**

### Exercise 5

To answer this question, you will need to edit the OSeMOSYS parameter file which defines the price of resources.  This is `VariableCost`. Remember that you can use the `edit_data()` function to change the values. Run the cell below to check the current `VariableCost` data.

In [ ]:
show_data(model='gas_more', dataset='VariableCost')

Now change the relevant values in the next cell, run the model againand generate the plots above.

In [ ]:
edit_data(model='gas_more',
          dataset='VariableCost',
          technology='', # Complete this
          value=) # Complete this

edit_data(model='gas_more',
          dataset='VariableCost',
          technology='', # Complete this
          value=) # Complete this


**Now, answer quiz question 5 on Canvas**

### Exercise 6

To answer this question, you will need to edit the OSeMOSYS parameter files which defines the efficiency of a technology. These are `InputActivityRatio` and `OutputActivityRatio`.

In [ ]:
show_data(model='gas_more', dataset='InputActivityRatio')

In [ ]:
show_data(model='gas_more', dataset='OutputActivityRatio')

Now edit the file

In [ ]:
edit_data(model='gas_more',
          dataset='', # Complete this
          technology='', # Complete this
          value=) # Complete this

**Now, answer quiz question 6 on Canvas**

## Summary of Stage 2

In stage 2 we learnt the following:

- Our computation of marginal costs using OSeMOSYS holds for a more complicated energy system structure
- The short-run marginal cost of electricity is a function of the cost of resources and efficiency of the conversion plants

<a name='s3'></a>
# Stage 3: A tax on pollution

In this stage, we'll now implement a financial penalty for emission of CO2.  

### Exercise 7

Before proceeding, please **answer quiz question 7 on Canvas**.

### Adding an emissions penalty to OSeMOSYS
To impose a financial penalty in OSeMOSYS we can edit the `EmissionsPenalty` parameter. Units are \$/tCO2.

You'll need to add a new line to the parameter file like so:

    SIMPLICITY,CO2,2014,50
    
First try a value of $50/tCO2.  You may use the plot below to compare your results with a zero emission price case (from the previous exercise).

In [ ]:
show_data('gas_more', 'EmissionsPenalty')

In [ ]:
add_data(model='gas_more', dataset='EmissionsPenalty',
         data={'REGION': 'SIMPLICITY', 'EMISSION': 'CO2', 'YEAR': 2014, 'VALUE': 50})

In [ ]:
emission_results = run_model("gas_more", 1, 60, 2)

In [ ]:
emissions_data = pd.DataFrame(emission_results)
emissions_data['co2_price'] = 50
emit_more_data = more_data.copy()
emit_more_data['co2_price'] = 0

all_data = pd.concat([emissions_data, emit_more_data], axis=0)
px.line(all_data, x='quantity', y='value', color='co2_price', facet_col='param',
        line_shape='vh',
        labels={'value': 'Cost', 'co2_price': 'CO2 Tax ($/tCO2)'})

### Exercise 8
Now please go to **quiz question 8 in Canvas**.

### Exercise 9
Please answer **quiz question 9 on Canvas**.

### Summary of Stage 3
In this stage, we have explored the effects of a carbon tax on the supply curves of gas, coal and final electricity. In the exercises on Canvas we have thought about the differences between carbon taxes and emission trading systems. Furthermore, we have analysed how a carbon price affects different fuels and how we can see these effects in the marginal cost curve of electricity.

<a name='s4'></a>
# Stage 4: Renewable electricity
In this stage we are adding a solar PV technology and a wind power technology to our model. We are now working with the model in `model/gas_solar_wind`. In the following we want to explore and think about the effects that the addition of solar PV and Wind have on the modelled system.
![](https://github.com/KTH-dESA/MJ2383/blob/main/img/gassolarwind.svg?raw=1)

The solar PV technology (`SOLPV`) and wind turbine technology (`WIND`) have the following characteristics:

Plant | Parameter | Value
:--|:--|--:
SOLPV | CapitalCost | 1700.0
WIND | CapitalCost | 1845.0
SOLPV | VariableCost | 2.5
WIND| VariableCost |4.17
SOLPV | OutputActivityRatio(FEL) | 1.0
WIND | OutputActivityRatio(SEC_EL) | 1.0
SOLPV | ResidualCapacity | 2
WIND| ResidualCapacity |3

Solar PV and Wind have the following availability defined in the file `CapacityFactor.csv`:

Timeslice | SOLPV | WIND
:--|:--|--:
ID | 0.3 | 0.25
IN | 0.0 | 0.25
SD | 0.4 | 0.25
SN | 0.0 | 0.25
WD | 0.25 | 0.25
WN | 0.0 | 0.25

You don't have to change anything.

### Exercise 10
**Before** running our new model, please think about the effect that solar PV and wind power will have on the marginal cost curves that we looked at in the previous stages. And please **answer quiz question 10 on Canvas**.

In [ ]:
results = run_model("gas_solar_wind", 1, 60, 2)

In [ ]:
renewable_data = pd.DataFrame(results)
px.line(renewable_data, x='quantity', y='value', facet_col='param', range_x=[0,65], range_y=[0,45], line_shape='vh')

In [ ]:
production = pd.read_csv('results/ProductionByTechnologyAnnual.csv').groupby(by=['TECHNOLOGY', 'FUEL']).sum()
px.bar(production.reset_index(), x='FUEL', y='VALUE', color='TECHNOLOGY')

### Exercise 11
Now please go to **quiz question 11 on Canvas**.

### Summary of Stage 4
In this stage we explored the effects that renewable power generation technologies have on the marginal cost curve of electricity. We discovered that the introduction of renewable power sources:
- does not necessarily push the technologies with the highest emissions out of the market
- from a operational point of view a market does not necessarily create a well functioning technology mix, e.g. the combination of coal power with renewable technologies creates operational challenges due to the inertia of coal fired power plants

# Extension Tasks

If you finish during the lab, you might want to complete one of the following optional tasks:

- Return to stage 2 and add a wind or solar technology into the OSeMOSYS model stored in `model/gas_more/data`.  You'll need to edit the following files:
  - `InputActivityRatio.csv`,
  - `OutputActivityRatio.csv`,
  - `CapacityFactor.csv`
  - `OperationalLife.csv`,
  - `ResidualCapacity.csv`,
  - `TECHNOLOGY.csv`,
  - `FixedCost.csv` and
  - `VariableCost.csv`.

  Remember that you can add data to the files using the `add_data(model, dataset, data)` function (see how we added the emission penalty previously).
  
  Once you've implemented the renewable technology, observe what happens to the supply cost curve.
  
  
- Return to stage 4 and add an `EmissionsPenalty` to the model, observe what happens to the supply cost curve depending on the hight of penalty you implement.

# Summary of lab 3
In this lab we have explored the following:
- the behaviour of marignal cost curves of different fuels in an OSeMOSYS model in different system settings
- the functioning of electricity markets and their sensitivity to price changes
- the possibility to extract marginal costs from OSeMOSYS models
- the use of a range of parameters to define a power system in OSeMOSYS